In [1]:
import numpy as np
import pandas as pd
from glob import glob
import sys
import os
from datetime import datetime
import logging

import matplotlib.pyplot as plt


from decimal import Decimal
from tinkoff.invest import MoneyValue


sys.path.append("..") 
from tp_utils import *
from tp_config import *
from tp_assets import *
import tink_port as tink


pd.set_option("display.max_rows", 90)

In [2]:
TOKEN0 = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
TOKEN1 = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'
SANDBOX = 't.qTfMeDk8iM5GLjIGj5Q5DVSnGdvOmSOzG4r3jQqdkdE2YUJMtFvBNb4v-Tyr50-4rxPBqia2jT-kBsE4NtoiKw'
token = SANDBOX
TOKEN = token

## Tinkoff sandbox

#### Открываем новый счет

In [3]:
logging.basicConfig(format="%(asctime)s %(levelname)s:%(message)s", level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [4]:
from tinkoff.invest.sandbox.client import SandboxClient

"""Example - How to set/get balance for sandbox account.
How to get/close all sandbox accounts.
How to open new sandbox account."""
with SandboxClient(token) as client:
    # get all sandbox accounts
    sandbox_accounts = client.users.get_accounts()
    print(sandbox_accounts)

    # close all sandbox accounts
    for sandbox_account in sandbox_accounts.accounts:
        client.sandbox.close_sandbox_account(account_id=sandbox_account.id)

    # open new sandbox account
    sandbox_account = client.sandbox.open_sandbox_account()
    print(sandbox_account.account_id)

    account_id = sandbox_account.account_id

2024-04-03 09:04:58,402 INFO:3eaa3e50d3bb822d04be2667986ed087 GetAccounts
2024-04-03 09:04:58,446 INFO:b96a2cf3e72fb0c49bc58788c01e15f2 CloseSandboxAccount
2024-04-03 09:04:58,514 INFO:ee7256ae8023d14530c6c69b9a9645d5 OpenSandboxAccount


GetAccountsResponse(accounts=[Account(id='c5bfa99f-9617-4f84-a3f6-6a463e50c494', type=<AccountType.ACCOUNT_TYPE_TINKOFF: 1>, name='', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2024, 4, 3, 4, 54, 36, 775257, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_FULL_ACCESS: 1>)])
0ba3bb27-0fe2-4990-9dde-46d971815a8a


#### Add money to Account

In [5]:
from tinkoff.invest.utils import decimal_to_quotation, quotation_to_decimal

def add_money_sandbox(client, account_id, money, currency="rub"):
    """Function to add money to sandbox account."""
    money = decimal_to_quotation(Decimal(money))
    return client.sandbox.sandbox_pay_in(
        account_id=account_id,
        amount=MoneyValue(units=money.units, nano=money.nano, currency=currency),
    )

with SandboxClient(token) as client:
      # add initial 2 000 000 to sandbox account
        print(add_money_sandbox(client=client, account_id=account_id, money=2000000))
        logger.info(
            "positions: %s", client.operations.get_positions(account_id=account_id)
        )
        print(
            "money: ",
            float(
                quotation_to_decimal(
                    client.operations.get_positions(account_id=account_id).money[0]
                )
            ),
        )

2024-04-03 09:05:02,471 INFO:54671f68a199ba4a380bdc86db933332 SandboxPayIn
2024-04-03 09:05:02,506 INFO:58fbb9bf8029fdf802b88d069a032352 GetPositions
2024-04-03 09:05:02,506 INFO:positions: PositionsResponse(money=[MoneyValue(currency='rub', units=2000000, nano=0)], blocked=[MoneyValue(currency='rub', units=0, nano=0)], securities=[], limits_loading_in_progress=False, futures=[], options=[])
2024-04-03 09:05:02,539 INFO:7dc6e8ce353e52f06801d751b2fb165b GetPositions


SandboxPayInResponse(balance=MoneyValue(currency='rub', units=2000000, nano=0))
money:  2000000.0


#### Покупаем базовый портфель

In [6]:
base = tink.get_id_base(token)

# BBG004730ZJ9 - VTBR / BBG004730N88 - SBER
portfolio = {'VTBR':10, 'POSI':20, 'RTKM':30, 'IRKT':20}

for asset in portfolio:
    figi = tink.ticker_to_figi(asset, base)
    print(figi)
    

2024-04-03 09:06:31,581 INFO:472afbe5762b79d1840694e28d7a6db7 Shares
2024-04-03 09:06:32,664 INFO:44bb2e53ae30b2053779fdc67c125184 Currencies
2024-04-03 09:06:32,796 INFO:6ff7119d76384cfad55b23c7548deb62 Futures
2024-04-03 09:06:33,127 INFO:e2f98a58f1238069cb6469792c98c38c Bonds
2024-04-03 09:06:33,932 INFO:f62cd080170f981e3e2320dcbd12c706 Etfs


BBG004730ZJ9
TCS00A103X66
BBG004S682Z6
BBG000FWGSZ5


In [82]:
base[base.lot > 1];

In [7]:
from tinkoff.invest import OrderDirection, OrderType


with SandboxClient(token) as client:
    for asset in portfolio:
        print(asset)
        figi = tink.ticker_to_figi(asset, base)
        qty = portfolio[asset]
        resp = client.orders.post_order(figi=figi,
                    quantity=qty,
                    direction=OrderDirection.ORDER_DIRECTION_BUY,
                    account_id=account_id,
                    order_type=OrderType.ORDER_TYPE_MARKET,)

2024-04-03 09:06:38,382 ERROR:e7da5cd8100d1edf5fd00401c2985ee1 PostOrder INVALID_ARGUMENT 30079


VTBR


RequestError: (<StatusCode.INVALID_ARGUMENT: (3, 'invalid argument')>, '30079', Metadata(tracking_id='e7da5cd8100d1edf5fd00401c2985ee1', ratelimit_limit='200, 200;w=60', ratelimit_remaining=199, ratelimit_reset=23, message='instrument is not available for trading'))

### Account Operations

In [29]:
with SandboxClient(token) as client:
    logger.info("orders: %s", client.orders.get_orders(account_id=account_id))
    logger.info(
        "positions: %s", client.operations.get_positions(account_id=account_id)
    )
    logger.info(
        "portfolio: %s", client.operations.get_portfolio(account_id=account_id)
    )
    logger.info(
        "operations: %s",
        client.operations.get_operations(
            account_id=account_id,
            from_=datetime(2023, 1, 1),
            to=datetime(2023, 2, 5),
        ),
    )
    logger.info(
        "withdraw_limits: %s",
        client.operations.get_withdraw_limits(account_id=account_id),
    )

2024-04-03 09:01:50,439 INFO:06c39aebd0eb9e6e7d5b4fac902a13b8 GetOrders
2024-04-03 09:01:50,440 INFO:orders: GetOrdersResponse(orders=[])
2024-04-03 09:01:50,474 INFO:9317a534af5f7646e3d7a2efc8033bcb GetPositions
2024-04-03 09:01:50,475 INFO:positions: PositionsResponse(money=[MoneyValue(currency='rub', units=2000000, nano=0)], blocked=[MoneyValue(currency='rub', units=0, nano=0)], securities=[], limits_loading_in_progress=False, futures=[], options=[])
2024-04-03 09:01:50,525 INFO:1b9aa09257a00aece836613763656e1e GetPortfolio
2024-04-03 09:01:50,527 INFO:portfolio: PortfolioResponse(total_amount_shares=MoneyValue(currency='rub', units=0, nano=0), total_amount_bonds=MoneyValue(currency='rub', units=0, nano=0), total_amount_etf=MoneyValue(currency='rub', units=0, nano=0), total_amount_currencies=MoneyValue(currency='rub', units=2000000, nano=0), total_amount_futures=MoneyValue(currency='rub', units=0, nano=0), expected_yield=Quotation(units=0, nano=0), positions=[PortfolioPosition(figi=

### Закрытие аккаунта

In [94]:
with SandboxClient(token) as client:        
        # add + 2 000 000 to sandbox account, total is 4 000 000
        print(add_money_sandbox(client=client, account_id=account_id, money=2000000))
        logger.info(
            "positions: %s", client.operations.get_positions(account_id=account_id)
        )

        # close new sandbox account
        sandbox_account = client.sandbox.close_sandbox_account(
            account_id=sandbox_account.account_id
        )
        print(sandbox_account)


2024-04-03 08:49:31,164 INFO:e1c2e00fc9df01e394c402f3586dce97 SandboxPayIn
2024-04-03 08:49:31,216 INFO:9d125879de4725001d59b9e069895974 GetPositions
2024-04-03 08:49:31,217 INFO:positions: PositionsResponse(money=[MoneyValue(currency='rub', units=4000000, nano=0)], blocked=[MoneyValue(currency='rub', units=0, nano=0)], securities=[], limits_loading_in_progress=False, futures=[], options=[])
2024-04-03 08:49:31,263 INFO:6bf3f26705a8f48ba201c371bfa994f5 CloseSandboxAccount


SandboxPayInResponse(balance=MoneyValue(currency='rub', units=4000000, nano=0))
CloseSandboxAccountResponse()


### Get order price

In [28]:

import logging
import os
from decimal import Decimal

from tinkoff.invest import Client, GetOrderPriceRequest, OrderDirection
from tinkoff.invest.utils import decimal_to_quotation

TOKEN = TOKEN1

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


INSTRUMENT_ID = "TCS00A105GE2"
QUANTITY = 1
PRICE = 230.1



logger.info("Getting Max Lots")
with SandboxClient(TOKEN) as client:
 #   response = client.users.get_accounts()
 #   account, *_ = response.accounts
 #   account_id = account.id

    logger.info(
        "Get pre-trade order commission and price for instrument=%s, quantity=%s and price=%s",
        INSTRUMENT_ID,
        QUANTITY,
        PRICE,
    )
    get_order_price = client.orders.get_order_price(
        GetOrderPriceRequest(
            account_id=account_id,
            instrument_id=INSTRUMENT_ID,
            quantity=QUANTITY,
            direction=OrderDirection.ORDER_DIRECTION_BUY,
            price=decimal_to_quotation(Decimal(PRICE)),
        )
    )

print(get_order_price)


2024-04-03 08:59:44,571 INFO:Getting Max Lots
2024-04-03 08:59:44,573 INFO:Get pre-trade order commission and price for instrument=TCS00A105GE2, quantity=1 and price=230.1
2024-04-03 08:59:44,724 ERROR:b04d0fb6967116ac629eb43dcea83191 GetOrderPrice INTERNAL 70001


RequestError: (<StatusCode.INTERNAL: (13, 'internal')>, '70001', Metadata(tracking_id='b04d0fb6967116ac629eb43dcea83191', ratelimit_limit='200, 200;w=60', ratelimit_remaining=199, ratelimit_reset=16, message='internal error'))

In [41]:
account_id

'9da21a29-1529-4b06-8e5f-7d88cca949c3'

In [24]:
import logging
import os
from decimal import Decimal

from tinkoff.invest import (
    Client,
    InstrumentIdType,
    StopOrderDirection,
    StopOrderExpirationType,
    StopOrderType,
)
from tinkoff.invest.exceptions import InvestError
from tinkoff.invest.utils import decimal_to_quotation, quotation_to_decimal

with SandboxClient(TOKEN) as client:
#    response = client.users.get_accounts()
#    account, *_ = response.accounts
#    account_id = account.id
    logger.info("Orders: %s", client.orders.get_orders(account_id=account_id))

    figi = "BBG004730ZJ9"  # BBG004730ZJ9 - VTBR / BBG004730N88 - SBER

    # getting the last price for instrument
    last_price = (
        client.market_data.get_last_prices(figi=[figi]).last_prices[0].price
    )
    last_price = quotation_to_decimal(last_price)
    print(f"figi, last price = {last_price}")

    # setting the percentage by which the takeprofit stop order
    # should be set below the last price
    percent_down = 5

    # calculation of the price for takeprofit stop order
    calculated_price = last_price - last_price * Decimal(percent_down / 100)
    print(f"calculated_price = {calculated_price}")

    # getting the min price increment and the number of digits after point
    min_price_increment = client.instruments.get_instrument_by(
        id_type=InstrumentIdType.INSTRUMENT_ID_TYPE_FIGI, id=figi
    ).instrument.min_price_increment
    number_digits_after_point = 9 - len(str(min_price_increment.nano)) + 1
    min_price_increment = quotation_to_decimal(min_price_increment)
    print(
        f"min_price_increment = {min_price_increment}, "
        f"number_digits_after_point={number_digits_after_point}"
    )

    # calculation of the price for instrument which is
    # divisible to min price increment
    calculated_price = (
        round(calculated_price / min_price_increment) * min_price_increment
    )
    print(
        f"let's send stop order at price = "
        f"{calculated_price:.{number_digits_after_point}f} divisible to "
        f"min price increment {min_price_increment}"
    )


NameError: name 'TOKEN' is not defined

### Market Order

In [13]:
with SandboxClient(token) as client:
    trading_status = client.market_data.get_trading_status(
        figi=figi
    )
trading_status.api_trade_available_flag
trading_status.market_order_available_flag

2024-04-03 09:11:12,726 INFO:0ededbd580708669f87e75db1eda44f8 GetTradingStatus


False

In [9]:
from tinkoff.invest import OrderDirection, OrderType
figi = "BBG004730ZJ9"
with SandboxClient(token) as client:
    resp = client.orders.post_order( figi=figi,
                quantity=1,
                direction=OrderDirection.ORDER_DIRECTION_BUY,
                account_id=account_id,
                order_type=OrderType.ORDER_TYPE_MARKET,)

2024-04-03 09:07:12,160 ERROR:18390a18d80f59ef1d8b0a6b70e7a0e6 PostOrder INVALID_ARGUMENT 30079


RequestError: (<StatusCode.INVALID_ARGUMENT: (3, 'invalid argument')>, '30079', Metadata(tracking_id='18390a18d80f59ef1d8b0a6b70e7a0e6', ratelimit_limit='200, 200;w=60', ratelimit_remaining=199, ratelimit_reset=49, message='instrument is not available for trading'))

In [55]:
resp

PostOrderResponse(order_id='8b666896-b27d-46e1-bcec-435b5d5f6231', execution_report_status=<OrderExecutionReportStatus.EXECUTION_REPORT_STATUS_FILL: 1>, lots_requested=1, lots_executed=1, initial_order_price=MoneyValue(currency='rub', units=232, nano=0), executed_order_price=MoneyValue(currency='rub', units=0, nano=23200000), total_order_amount=MoneyValue(currency='rub', units=232, nano=0), initial_commission=MoneyValue(currency='rub', units=0, nano=116000000), executed_commission=MoneyValue(currency='rub', units=0, nano=116000000), aci_value=MoneyValue(currency='', units=0, nano=0), figi='BBG004730ZJ9', direction=<OrderDirection.ORDER_DIRECTION_BUY: 1>, initial_security_price=MoneyValue(currency='rub', units=0, nano=23200000), order_type=<OrderType.ORDER_TYPE_MARKET: 2>, message='', initial_order_price_pt=Quotation(units=0, nano=0), instrument_uid='8e2b0325-0292-4654-8a18-4f63ed3b0e09', order_request_id='', response_metadata=ResponseMetadata(tracking_id='d4f88ec0baaae61b8b3fd5692828b8

In [66]:
base = tink.get_id_base(token)

sess = SandboxClient(token) 

with sess as cl:
    port = cl.operations.get_portfolio(account_id=account_id) 
tink.port_to_df(port, base)

2024-04-03 08:22:30,915 INFO:3b660782159cb02542c10cdfa5550706 Shares
2024-04-03 08:22:32,176 INFO:27fa9439f271f783e285ff053e0a4076 Currencies
2024-04-03 08:22:32,436 INFO:52f29eda572f433d4b4eac4b7b452298 Futures
2024-04-03 08:22:33,798 INFO:c85b8fa40e8996550b72672ee3c5b185 Bonds
2024-04-03 08:22:34,731 INFO:8f26771087d22975c10be9585acb9c0b Etfs
2024-04-03 08:22:35,041 INFO:1b446e3e56694a780af05eb98b175fa5 GetPortfolio


,figi,ticker,name,quantity,price
1,BBG004730ZJ9,VTBR,Банк ВТБ,20000,0.02363
0,RUB000UTSTOM,None,None,1999535,0.00000


In [53]:
with SandboxClient(token) as client:
    statuses = client.market_data.get_trading_statuses(instrument_ids=["BBG004730N88"])
    print(statuses)

2024-04-02 20:12:51,913 INFO:a5543646ba0e0aea5be67619ae261c2a GetTradingStatuses


GetTradingStatusesResponse(trading_statuses=[GetTradingStatusResponse(figi='BBG004730N88', trading_status=<SecurityTradingStatus.SECURITY_TRADING_STATUS_NORMAL_TRADING: 5>, limit_order_available_flag=True, market_order_available_flag=True, api_trade_available_flag=True, instrument_uid='e6123145-9665-43e0-8413-cd61b8aa9b13', bestprice_order_available_flag=True, only_best_price=False)])


## Helper functions

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

from datetime import datetime
now = datetime.today().strftime('%Y-%m-%d')


In [16]:
def calculate_portfolio_difference(old_portfolio, new_portfolio):
    """
    Рассчитать разницу между двумя портфелями.

    Args:
        old_portfolio: Словарь, где ключом является тикер, а значением - количество акций.
        new_portfolio: Словарь, где ключом является тикер, а значением - количество акций.

    Returns:
        Словарь, где ключом является тикер, а значением - разница между количеством акций в 
        новых и старых портфелях.
    """

    difference = {}
    for ticker in new_portfolio:
        if ticker in old_portfolio:
            difference[ticker] = new_portfolio[ticker] - old_portfolio[ticker]
        else:
            difference[ticker] = new_portfolio[ticker]

    for ticker in old_portfolio:
        if ticker not in new_portfolio:
            difference[ticker] = -old_portfolio[ticker]

    return difference


In [31]:
def calculate_portfolio_allocation(weights, total_amount, lot_sizes):
    """
    Рассчитать распределение суммы на веса в портфеле, учитывая лоты.

    Args:
        weights: Список весов акций в портфеле.
        total_amount: Общая сумма, которую нужно распределить.
        lot_sizes: Список размеров лотов для акций.

    Returns:
        Список, где каждый элемент - количество акций, 
        округленное до размера лота.
    """

    # Нормализация весов
    weights = normalize_weights(weights)

    # Распределение суммы
    amounts = [weight * total_amount for weight in weights]

    # Округление до размера лота
    quantities = [round(amount / lot_size, 0) for amount, lot_size in zip(amounts, lot_sizes)]

    # Корректировка с учетом минимального лота
    for i in range(len(quantities)):
        if lot_sizes[i] > 0 and quantities[i] * lot_sizes[i] < total_amount:
            quantities[i] = 1

    # Распределение остатка
    remaining_amount = total_amount - sum(quantity * lot_size for quantity, lot_size in zip(quantities, lot_sizes))

    return quantities, remaining_amount


def normalize_weights(weights):
    """
    Нормализовать веса, чтобы они давали в сумме единицу.

    Args:
        weights: Список весов.

    Returns:
        Список нормализованных весов.
    """

    total_weight = sum(weights)
    return [weight / total_weight for weight in weights]


# Пример
weights = [0.4, 0.3, 0.3]
total_amount = 10000
lot_sizes = [1, 10, 1000]

quantities, rem = calculate_portfolio_allocation(weights, total_amount, lot_sizes)

# Вывод
for quantity, weight in zip(quantities, weights):
    print(f"Количество: {quantity}, Вес: {weight:.2%}")

print(f"\nОбщая сумма: {sum(quantity * lot_size for quantity, lot_size in zip(quantities, lot_sizes))}")

Количество: 1, Вес: 40.00%
Количество: 1, Вес: 30.00%
Количество: 1, Вес: 30.00%

Общая сумма: 1011


In [21]:
def calculate_portfolio_allocation(weights, total_amount, lot_sizes):
    """
    Рассчитать распределение суммы на веса в портфеле, учитывая лоты.

    Args:
        weights: Список весов акций в портфеле.
        total_amount: Общая сумма, которую нужно распределить.
        lot_sizes: Список размеров лотов для акций.

    Returns:
        Список, где каждый элемент - 
        словарь с информацией о количестве акций, цене за 1 акцию, 
        цене за лот и стоимости позиции.
    """

    # Нормализация весов
    weights = normalize_weights(weights)

    # Распределение суммы
    amounts = [weight * total_amount for weight in weights]

    # Округление до размера лота
    quantities = [round_to_lot_size(amount, lot_size) for amount, lot_size in zip(amounts, lot_sizes)]

    # Расчет стоимости позиции
    position_values = [quantity * lot_size for quantity, lot_size in zip(quantities, lot_sizes)]

    # Формирование выходных данных
    portfolio = []
    for quantity, lot_size, position_value in zip(quantities, lot_sizes, position_values):
        portfolio.append({
            "quantity": quantity,
            "price_per_lot": position_value / quantity,
            "position_value": position_value,
        })

    return portfolio


def round_to_lot_size(amount, lot_size):
    """
    Округлить число до размера лота.

    Args:
        amount: Число, которое нужно округлить.
        lot_size: Размер лота.

    Returns:
        Округленное число.
    """

    if lot_size == 1:
        return round(amount)

    return round(amount / lot_size) * lot_size


# Пример
weights = [0.4, 0.3, 0.3]
total_amount = 10000
lot_sizes = [1, 10, 1000]

portfolio = calculate_portfolio_allocation(weights, total_amount, lot_sizes)

# Вывод
for position in portfolio:
    print(f"Количество: {position['quantity']}")
    print(f"Цена за лот: {position['price_per_lot']}")
    print(f"Стоимость позиции: {position['position_value']}")
    print()

print(f"\nОбщая сумма: {sum(position['position_value'] for position in portfolio)}")

Количество: 4000
Цена за лот: 1.0
Стоимость позиции: 4000

Количество: 3000
Цена за лот: 10.0
Стоимость позиции: 30000

Количество: 3000
Цена за лот: 1000.0
Стоимость позиции: 3000000


Общая сумма: 3034000
